In [2]:
import numpy as np
import pandas as pd

In [2]:
import os
os.listdir('data')

['nemu_inputs',
 'Bloom_etal',
 'species.csv',
 'trees',
 'vir_linages.csv',
 'virus_taxid_host_gene.csv',
 'nemu',
 'flt',
 'cossim_nemu_bloom.csv',
 'ms12syn_all_virus.csv',
 'taxid_virus_type.csv',
 'viral_spectra_dataset.csv',
 'allmut_nemu.zip',
 'allmut_nemu.csv',
 '.gitignore',
 'virs_fit_metrics.csv',
 'virs_rnd_fit_metrics.csv',
 'aminoacid_eq_freq_all_virus.csv',
 'aminoacid_eq_freq_sars_cov2.csv',
 'losers_gainers.csv',
 'nucl_eq_freq_all_virus.csv',
 'aminoacid_freq_all_virus.csv',
 'df_taxid_ref_seq.csv',
 'nucl_4f_freq_all_virus.csv',
 'nucl_all_freq_all_virus.csv',
 'nucl_eq_freq_sars_cov2.csv',
 'mutation_summary_vir.csv',
 'nemu_results_qc.csv']

In [7]:
spectra = pd.read_csv('data/viral_spectra_dataset.csv')#.query('df == "nemu"')
raw_spectra = pd.read_csv('data/ms12syn_all_virus.csv')
host_gene = pd.read_csv('data/virus_taxid_host_gene.csv')
vir_type = pd.read_csv('data/taxid_virus_type.csv', index_col=0)
linages = pd.read_csv('data/vir_linages.csv')
seq_cnt = pd.read_csv('data/nemu_inputs/seq_cnt.csv') # 3052310(11620),156,7587

# vir_type['taxid'] = vir_type['taxid'].astype(int)

In [23]:
raw_spectra.taxid.unique()

array([  10407,   10990,   11082,   11320, 1133363, 1157337,  118655,
         11983,   12092,   12110,   12162,   12637,  138948,  138950,
        138951,  162145, 1678143, 1933178,  198112,   28295,   28344,
       3052230, 3052493, 3052763,  351073,   38170,   40054,   54290,
         57482,  693997,  694014,   77763])

In [8]:
final_df = vir_type[vir_type.taxid.isin(raw_spectra.taxid.unique())]\
    .merge(linages, how='left')\
        .merge(seq_cnt, how='left', on='taxid')
final_df.head()

,taxid,species,host,gene,comment,Type,genus,family,order,class,phylum,kingdom,superkingdom,Nucleotide_Count,sequences,nucleotide_sites
0,38170,Avian orthoreovirus__38170,NaN,NaN,NaN,ds,Orthoreovirus__10882,Reoviridae__10880,Reovirales__2732541,Resentoviricetes__2732459,Duplornaviricota__2732405,Orthornavirae__2732396,Viruses__10239,4363.0,199.0,4011.0
1,1157337,Piscine orthoreovirus__1157337,NaN,NaN,NaN,ds,Orthoreovirus__10882,Reoviridae__10880,Reovirales__2732541,Resentoviricetes__2732459,Duplornaviricota__2732405,Orthornavirae__2732396,Viruses__10239,2478.0,79.0,4049.0
2,351073,Mammalian orthoreovirus__351073,NaN,NaN,NaN,ds,Orthoreovirus__10882,Reoviridae__10880,Reovirales__2732541,Resentoviricetes__2732459,Duplornaviricota__2732405,Orthornavirae__2732396,Viruses__10239,2366.0,161.0,3968.0
3,40054,Epizootic hemorrhagic disease virus__40054,NaN,NaN,NaN,ds,Orbivirus__10892,Reoviridae__10880,Reovirales__2732541,Resentoviricetes__2732459,Duplornaviricota__2732405,Orthornavirae__2732396,Viruses__10239,1773.0,NaN,NaN
4,10990,Rice black streaked dwarf virus__10990,Avena sativa,P1 protein,NaN,ds,Fijivirus__10988,Reoviridae__10880,Reovirales__2732541,Resentoviricetes__2732459,Duplornaviricota__2732405,Orthornavirae__2732396,Viruses__10239,1149.0,123.0,4395.0


In [9]:
final_df.shape

(32, 16)

In [10]:
# species names do not match taxids in some cases. Comment column store this info
final_df[final_df['species'].str.extract('.+__(.+)').astype(int)[0] != final_df['taxid']]

,taxid,species,host,gene,comment,Type,genus,family,order,class,phylum,kingdom,superkingdom,Nucleotide_Count,sequences,nucleotide_sites
9,3052493,Puumala orthohantavirus__1980486,Myodes glareolus,polymerase,oldTaxid:1980486,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.0,6471.0
10,118655,Oropouche orthobunyavirus__1933309,Homo sapiens,RNA polymerase,OldTaxid:1933309,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,472.0,6760.0
11,3052763,Rice stripe tenuivirus__12331,NaN,NaN,OldTaxid:12331,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,1133363,Schmallenberg orthobunyavirus__2560743,Ovis aries,M polyprotein,OldTaxid:2560743,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118.0,4221.0
28,54290,Pegivirus C__1985356,Homo sapiens,polyprotein,OldTaxid:1985356,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,8551.0


In [11]:
final_df['species'] = final_df['species'].str.replace('__.+', '', regex=True)

In [12]:
final_df.to_csv('data/viral_taxid_info_be.csv', index=False)

In [16]:
raw_spectra.query('taxid == 10407')

,Mut,ObsNum,ExpNum,MutSpec,MutSpec_q05,MutSpec_median,MutSpec_q95,taxid
0,A>C,222.1000,109.4550,0.109127,0.098138,0.109519,0.118038,10407
1,A>G,348.5810,159.7600,0.117342,0.109575,0.117365,0.127528,10407
2,A>T,114.4490,83.7826,0.073464,0.061877,0.074088,0.084475,10407
3,C>A,172.6090,154.1570,0.060217,0.053061,0.060244,0.065731,10407
4,C>G,28.6549,131.8260,0.011690,0.009010,0.012031,0.014995,10407
5,C>T,716.8470,257.9570,0.149450,0.142104,0.150746,0.158789,10407
6,G>A,523.6170,143.1970,0.196651,0.184005,0.195787,0.209227,10407
7,G>C,41.0114,81.7845,0.026968,0.020393,0.027325,0.032978,10407
8,G>T,31.5994,81.7845,0.020779,0.015946,0.020455,0.026900,10407
9,T>A,106.6500,155.3000,0.036932,0.030765,0.036855,0.041759,10407
